In [36]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import JSONSearchTool
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from typing import List
import asyncio

from dotenv import dotenv_values
from dotenv import load_dotenv
import os
#from crewai_tools import SerperDevTool
import requests
import json


In [37]:
OUTPUT_DIR = "./data_out/"

In [38]:
# load .env file to environment
load_dotenv()

config = dotenv_values("../.env")

# Set up API keys (replace with your actual API keys)
os.environ["OPENAI_API_KEY"] = config['OPENAI_API_KEY']
os.environ["ANTHROPIC_API_KEY"] = config['ANTHROPIC_API_KEY']
#os.environ["GOOGLE_API_KEY"] = config['GOOGLE_API_KEY']
#os.environ["SERPER_API_KEY"] = config['SERPER_API_KEY']

#If using env vars
#OPENAI_API_KEY=config['OPENAI_API_KEY']
#ANTHROPIC_API_KEY=config['ANTHROPIC_API_KEY']
#GOOGLE_API_KEY=config['GOOGLE_API_KEY']

# Initialize the language model
#llm = ChatOpenAI(temperature=0)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

In [39]:
#==================================================
# Define the tools
#==================================================

def search_nvd(keyword, max_results=3):
    base_url = "https://services.nvd.nist.gov/rest/json/cves/2.0"
    params = {
        "keywordSearch": keyword,
        "resultsPerPage": max_results
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        
        results = []
        for vuln in data.get('vulnerabilities', []):
            cve = vuln['cve']
            results.append({
                'id': cve['id'],
                'description': cve['descriptions'][0]['value'] if cve['descriptions'] else 'No description available',
                'published': cve['published'],
                'lastModified': cve['lastModified']
            })
        
        return results
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
search_query = "improper validation of string input"
results = search_nvd(search_query)

print(f"Search results for '{search_query}':")
for result in results:
    print(f"\n{result['id']}" ": " f"Description: {result['description']}")
    #print(f"\nID: {result['id']}")
    #print(f"Description: {result['description']}")
    #print(f"Published: {result['published']}")
    #print(f"Last Modified: {result['lastModified']}")
    
    
    


Search results for 'improper validation of string input':

CVE-2017-12243: Description: A vulnerability in the Cisco Unified Computing System (UCS) Manager, Cisco Firepower 4100 Series Next-Generation Firewall (NGFW), and Cisco Firepower 9300 Security Appliance could allow an authenticated, local attacker to obtain root shell privileges on the device, aka Command Injection. The vulnerability is due to improper validation of string input in the shell application. An attacker could exploit this vulnerability through the use of malicious commands. A successful exploit could allow the attacker to obtain root shell privileges on the device. Cisco Bug IDs: CSCvf20741, CSCvf60078.

CVE-2019-1831: Description: A vulnerability in the email message scanning of Cisco AsyncOS Software for Cisco Email Security Appliance (ESA) could allow an unauthenticated, remote attacker to bypass configured content filters on the device. The vulnerability is due to improper input validation of the email body. An

In [40]:
#==================================================
# Prompts
#==================================================

# Function to read the system prompt from a file
def read_system_prompt(file_path: str) -> str:
    with open(file_path, 'r') as file:
        return file.read()


keyphrases_extractor_prompt = read_system_prompt('../prompts/extract_key_entities/system.md')
assign_cwe_prompt = read_system_prompt('../prompts/assign_cwes/system.md')


In [41]:

#==================================================
# Define the agents
#==================================================

keyphrases_extractor = Agent(
    role='Keyphrases Extractor',
    goal='Extract key phrases from vulnerability descriptions',
    backstory='You are an expert in cybersecurity language and can identify crucial elements in vulnerability descriptions.',
    allow_delegation=False,
    llm=llm,
    verbose=True,  # Enable verbose mode
    memory=True,   # Enable memory if needed
    system_prompt=keyphrases_extractor_prompt
)

cwe_observer = Agent(
    role='CWE Observer',
    goal='Find CVEs with similar weakness keyphrases from CWE Observed Examples',
    backstory='You have extensive knowledge of Common Weakness Enumeration (CWE) and can identify related CVEs.',
    allow_delegation=False,
    llm=llm,
    verbose=True,  # Enable verbose mode
    memory=True,   # Enable memory if needed
)

top25_cwe_analyst = Agent(
    role='Top 25 CWE Analyst',
    goal='Identify CVEs with similar weakness keyphrases from Top 25 CWE Mappings',
    backstory='You specialize in the OWASP Top 25 Common Weakness Enumeration and can find relevant CVEs.',
    allow_delegation=False,
    llm=llm,
    verbose=True,  # Enable verbose mode
    memory=True,   # Enable memory if needed
)

nvd_researcher = Agent(
    role='NVD Researcher',
    goal='Discover CVEs with similar weakness keyphrases from the National Vulnerability Database',
    backstory='You are an expert in navigating the National Vulnerability Database and can find pertinent CVEs.',
    allow_delegation=False,
    llm=llm,
    verbose=True,  # Enable verbose mode
    memory=True,   # Enable memory if needed
)

report_creator = Agent(
    role='Report Creator',
    goal='Create a comprehensive vulnerability report',
    backstory='You are skilled in synthesizing information from various sources to create detailed cybersecurity reports.',
    allow_delegation=False,
    llm=llm,
    system_prompt=assign_cwe_prompt,
    verbose=True,  # Enable verbose mode
    memory=True,   # Enable memory if needed

)

report_reviewer = Agent(
    role='Report Reviewer',
    goal='Review and improve the vulnerability report',
    backstory='You have years of experience in reviewing and enhancing cybersecurity reports, ensuring they are accurate and actionable.',
    allow_delegation=False,
    llm=llm,
    verbose=True,  # Enable verbose mode
    memory=True,   # Enable memory if needed
)




#==================================================
# Define the tasks
#==================================================     

extract_key_phrases = Task(
    description=f"Extract key phrases from the following vulnerability description: {vulnerability_description}",
    expected_output="A list of key phrases that accurately represent the core aspects of the vulnerability.",
    output_file=os.path.join(OUTPUT_DIR, "key_phrases.txt"),
    agent=keyphrases_extractor,
)

find_similar_cves_from_cwes_observed = Task(
    description="Find CVEs with similar weakness keyphrases from CWE Observed Examples using the extracted key phrases.",
    expected_output="A list of relevant CVEs from CWE Observed Examples, including their IDs and brief descriptions.",
    async_execution=True,
    output_file=os.path.join(OUTPUT_DIR, "similar_cves_from_cwes_observed.txt"),
    agent=cwe_observer,
    context=[extract_key_phrases]
)

find_similar_cves_from_top25 = Task(
    description="Identify CVEs with similar weakness keyphrases from Top 25 CWE Mappings using the extracted key phrases.",
    expected_output="A list of relevant CVEs from Top 25 CWE Mappings, including their IDs, rankings, and brief descriptions.",
    async_execution=True,
    output_file=os.path.join(OUTPUT_DIR, "similar_cves_from_top25.txt"),
    agent=top25_cwe_analyst,
    context=[extract_key_phrases]
)

find_similar_cves_from_nvd = Task(
    description="Discover CVEs with similar weakness keyphrases from the National Vulnerability Database using the extracted key phrases.",
    expected_output="A comprehensive list of relevant CVEs from the NVD, including their IDs, CVSS scores, and brief descriptions.",
    async_execution=True,
    output_file=os.path.join(OUTPUT_DIR, "similar_cves_from_nvd.txt"),
    agent=nvd_researcher,
    context=[extract_key_phrases]
)

create_vulnerability_report = Task(
    description="Create a comprehensive report based on the vulnerability description, extracted key phrases, and identified CVEs from all sources.",
    expected_output="A detailed report that includes an overview of the vulnerability, analysis of related CVEs, potential impact, and initial recommendations for mitigation.",
    output_file=os.path.join(OUTPUT_DIR, "vulnerability_report_draft.txt"),
    agent=report_creator,
    context=[find_similar_cves_from_cwes_observed, find_similar_cves_from_top25, find_similar_cves_from_nvd]
)

review_vulnerability_report = Task(
    description="Review the created report, provide feedback, and improve it if necessary.",
    expected_output="A final, polished report that incorporates any necessary improvements, ensures accuracy, and provides actionable insights for addressing the vulnerability.",
    output_file=os.path.join(OUTPUT_DIR, "vulnerability_report_final.txt"),
    agent=report_reviewer,
)



SyntaxError: invalid syntax. Perhaps you forgot a comma? (4204407409.py, line 52)

In [ ]:

# Main function to run the vulnerability analysis
def analyze_vulnerability(vulnerability_description: str) -> str:
    tasks = create_tasks(vulnerability_description)
    vulnerability_analysis_crew = Crew(
        #agents=[keyphrases_extractor, cwe_observer, top25_cwe_analyst, nvd_researcher, report_creator, report_reviewer],
        #tasks=[extract_key_phrases, find_similar_cves_from_cwes_observed, find_similar_cves_from_top25, find_similar_cves_from_nvd, create_vulnerability_report,review_vulnerability_report],
        agents=[keyphrases_extractor],
        tasks=[extract_key_phrases],
        process=Process.sequential
    )
    result = vulnerability_analysis_crew.kickoff()
    return result

# Example usage
if __name__ == "__main__":
    #vulnerability_description = "A buffer overflow vulnerability in the XYZ software allows remote attackers to execute arbitrary code."
    vulnerability_description = "The Cisco Discovery Protocol implementation in Cisco IOS XR Software does not do improper validation of string input from certain fields which could allow an unauthenticated, adjacent attacker to execute arbitrary code or cause a reload on an affected device. The vulnerability is due to improper validation of string input from certain fields in Cisco Discovery Protocol messages. An attacker could exploit this vulnerability by sending a malicious Cisco Discovery Protocol packet to an affected device."
    #vulnerability_description = "The Widget Connector macro in Atlassian Confluence Server before version 6.6.12 (the fixed version for 6.6.x), from version 6.7.0 before 6.12.3 (the fixed version for 6.12.x), from version 6.13.0 before 6.13.3 (the fixed version for 6.13.x), and from version 6.14.0 before 6.14.2 (the fixed version for 6.14.x), allows remote attackers to achieve path traversal and remote code execution on a Confluence Server or Data Center instance via server-side template injection."
    #vulnerability_description = "The Popup Manager WordPress plugin through 1.6.6 does not have authorisation and CSRF check when creating/updating popups, and is missing sanitisation as well as escaping, which could allow unauthenticated attackers to create arbitrary popups and add Stored XSS payloads as well"
    
    final_report = analyze_vulnerability(vulnerability_description)
    print("Final Vulnerability Analysis Report:")
    print(final_report)
    
    
    #The Popup Manager WordPress plugin through 1.6.6 does not have authorisation and CSRF check when creating/updating popups, and is missing sanitisation as well as escaping, which could allow unauthenticated attackers to create arbitrary popups and add Stored XSS payloads as well
    #The Widget Connector macro in Atlassian Confluence Server before version 6.6.12 (the fixed version for 6.6.x), from version 6.7.0 before 6.12.3 (the fixed version for 6.12.x), from version 6.13.0 before 6.13.3 (the fixed version for 6.13.x), and from version 6.14.0 before 6.14.2 (the fixed version for 6.14.x), allows remote attackers to achieve path traversal and remote code execution on a Confluence Server or Data Center instance via server-side template injection.
    #The Cisco Discovery Protocol implementation in Cisco IOS XR Software does not do improper validation of string input from certain fields which could allow an unauthenticated, adjacent attacker to execute arbitrary code or cause a reload on an affected device. The vulnerability is due to improper validation of string input from certain fields in Cisco Discovery Protocol messages. An attacker could exploit this vulnerability by sending a malicious Cisco Discovery Protocol packet to an affected device. 

2024-09-07 14:53:31,847 - 129443974594880 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


Final Vulnerability Analysis Report:
**Vulnerability Report**

**Overview:**
The XYZ software is plagued by multiple buffer overflow vulnerabilities that can be exploited by remote attackers to execute arbitrary code on the target system. These vulnerabilities, identified through various sources and CVEs, pose a significant risk to the security and integrity of the software.

**Analysis of Related CVEs:**
1. CVE-2021-1234: This vulnerability allows remote attackers to execute arbitrary code via a crafted input, indicating a critical security flaw in the XYZ software.
2. CVE-2020-5678: Another buffer overflow vulnerability in XYZ software that can be exploited by remote attackers to execute arbitrary code, further highlighting the susceptibility of the software to such attacks.
3. CVE-2019-9999: This CVE also points to a buffer overflow vulnerability in XYZ software that enables remote attackers to execute arbitrary code on the target system.
4. CVE-2020-12345: With a high CVSS score of